So the last shall be first, and the first last: for many be called, but few chose

Matthew 20:16

In [1]:
import math
import pandas as pd
import seaborn as sns
from scipy import stats
sns.set()

# load libraries and set plot parameters - preambule
import numpy as np
import prettytable as pt
import matplotlib.pyplot as plt
%matplotlib inline
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('pdf', 'png')
plt.rcParams['savefig.dpi'] = 75
plt.rcParams['figure.autolayout'] = False
plt.rcParams['figure.figsize'] = 10, 6
plt.rcParams['axes.labelsize'] = 18
plt.rcParams['axes.titlesize'] = 20
plt.rcParams['font.size'] = 16
plt.rcParams['lines.linewidth'] = 2.0
plt.rcParams['lines.markersize'] = 8
plt.rcParams['legend.fontsize'] = 14
plt.rcParams['text.usetex'] = True
#plt.rcParams['font.family'] = 'serif'
#plt.rcParams['font.serif'] =  'cm'
#plt.rcParams['text.latex.preamble'] = usepackage{subdepth}, usepackage{type1cm}


figuresize = 3
titlefontsize = 8
titleweight = 'bold'

# Introdução

<img src='./database/logo_inc.jpg'>

* Instituto Nacional de Cardiologia:
    - O INC é referência no tratamento de doenças cardíacas de alta complexidade.  
    - Único hospital público no estado que realiza transplantes de cardíacos em adultos e crianças
    - É o segundo centro que mais realiza cirurgias de cardiopatias congênitas no Brasil.




      


## Infraestrututa:
  - 165 leitos (60 de UTI)
  - 4 mil internações anuais
  - 1200 cirurgias
  - 50 mil consultas médicas.
  
  
  
  <img src='./database/frente_inc.jpg'>
  
  

##  O Problema de Estoques

* Setor de Hemodinâmica:
    - O cateterismo cardíaco consiste em introduzir um cateter até o coração, através de uma artéria periférica.
    - Um cateter é introduzido e conduzido até o tronco das artérias coronárias esquerda e direita.
        - São procedimentos considerados mais simples (i.e: não são cirurgias)
        - Representam os procedimentos mais caros (Aproximadamente 3.8 milhões de reais nos últimos 2 anos)
        - A maioria das próteses custam de 15 até 25 mil reais
        
        

# Dados


In [2]:
BD_EQPT = pd.read_csv('./database/BD_Equipamentos.csv', encoding = 'utf8')

print("Quantidade de Itens Únicos:", BD_EQPT.shape[0])

BD_EQPT.head()


Quantidade de Itens Únicos: 235


,COD,MATERIAL,VALOR UNIT,QTDE,PRECO
0,53065,"PROTESE P/FECH.DEF.SEPTO INTERVENTR, CENTRO DE...","R$ 26,812.50",11,"$294,937.50"
1,52818,"PROT FECH SEPTO INTER ATRIAL, NUCLEO OCLUSOR E...","R$ 26,750.76",22,"$588,516.69"
2,55566,PROTESE PARA OCLUSAO DE DEFEITOS SEPTAIS ATRIA...,"R$ 25,200.00",8,"$201,600.00"
3,52816,"PROTESE P/FECH.DEF.SEPTO INTERATRIAL, NUCLEO E...","R$ 24,490.91",7,"$171,436.36"
4,52811,"PROTESE P/FECH.DEF.SEPTO ATRIAL, NUCLEO OCLUSO...","R$ 24,038.33",6,"$144,229.99"


* Hemodinâmica Pediátrica:
  - É o setor de hemodinâmica com os registros mais organizados
      - 441 atendimentos para 16 procedimentos
  
  (OBS: houve perda de 1/3 dos dados no cruzamento das informações )

<img src='./database/tabela_procedimentos.png'>

In [3]:
def cria_BD():
    BD_PED = pd.read_csv('./database/BD_CLEAN.csv')
    BD_PED['DATA_MOV'] = pd.to_datetime(BD_PED.DATA_MOV)
    BD_PED['PERIODO'] = BD_PED.DATA_MOV.dt.week
    
    
    first_year = BD_PED.DATA_MOV.dt.year[0]
    for row, year in zip(range(len(BD_PED)),BD_PED.DATA_MOV.dt.year):    
            BD_PED.loc[row, 'PERIODO']=BD_PED.loc[row, 'PERIODO']+(year - first_year)*52 
            
            
    BD_PED.loc[:, 'PERIODO']=BD_PED.loc[:, 'PERIODO']-BD_PED.loc[0, 'PERIODO']+1     
    BD_PED['PERIODO']
    
    return BD_PED

In [4]:
def BD_MAT_UNIT(BD_PED):
    
    Material_list = BD_PED.COD_MATERIAL.unique()
    BD_PED['UNIT'] = BD_PED.QTDE.min()
    mat_unit = {}
    for material in Material_list:
        BD_MAT = BD_PED[BD_PED.COD_MATERIAL == material]  
        if(BD_MAT.QTDE.max() > 5):
            if( (BD_MAT.QTDE.max()%BD_MAT.QTDE.min()) ==0):
                    mat_unit[material] = BD_MAT.QTDE.min()
        
    
    
    BD_PED.loc[BD_PED['COD_MATERIAL'] == 51287, 'QTDE'] = 10
    BD_PED.loc[BD_PED['COD_MATERIAL'] == 51287, 'UNIT'] = 10
    BD_PED.loc[9412, 'QTDE'] =5
    for mat in mat_unit:
            
        BD_PED.loc[BD_PED['COD_MATERIAL'] == mat, 'UNIT'] = mat_unit[mat]

               
    BD_PED['QTDE'] = BD_PED['QTDE']/BD_PED['UNIT']
    BD_PED['QTDE'] = BD_PED.QTDE.astype(int)
    return(BD_PED)


In [5]:
#Call part 2A
#Creating My database

BD_PED_ORIG = cria_BD()
BD_PED = BD_MAT_UNIT(BD_PED_ORIG)

print(BD_PED.QTDE.max())

print("Total de Itens:", BD_PED.shape[0])

BD_PED.head()


10
Total de Itens: 12307


,ATENDIMENTO,COD_PROCEDIMENTO,PROCEDIMENTO,COD_MATERIAL,ITEM,QTDE,DATA_MOV,PERIODO,UNIT
0,1264197,1,FECHAMENTO DE FENESTRAÇÃO,52811,"PROTESE P/FECH.DEF.SEPTO ATRIAL, NUCLEO OCLUSO...",1,2016-09-08,1,1
1,1264197,1,FECHAMENTO DE FENESTRAÇÃO,52168,INTRODUTOR SILICONE ATRAUMATICO 12F TIPO MULLINS,1,2016-09-08,1,1
2,1264197,1,FECHAMENTO DE FENESTRAÇÃO,54176,"FIO GUIA EXTRA RIGIDO 0,035"" COMPR. 260CM(TIPO...",1,2016-09-08,1,1
3,1264197,1,FECHAMENTO DE FENESTRAÇÃO,50842,"FIO GUIA EXTRA SUPORTE HIDROFILICO 0,014"" - 175CM",1,2016-09-08,1,1
4,1264197,1,FECHAMENTO DE FENESTRAÇÃO,56840,KIT CIRURGICO UNIVERSAL,1,2016-09-08,1,1


In [6]:
def Period_Setup():  
    
    Attendance_list = BD_PROC.ATENDIMENTO.unique()
    list_period = []
    for  att   in Attendance_list:
        BD_ATT = BD_PROC[BD_PROC.ATENDIMENTO == att]
        period = BD_ATT.PERIODO.min()
        list_period.append(period)
    
    
    Period_Frequency =  [list_period.count(x) for x in set(list_period)]
    Period_Frequency.sort()
    for i in range(1,max(BD_PED.PERIODO)+1):
        if(list_period.count(i)   ==0):
            Period_Frequency.append(0)
            
    #print(Period_Frequency)        
    return Period_Frequency

In [7]:
def dist_poisson(k, lamb):
    return (lamb**k/factorial(k)) * np.exp(-lamb)

In [8]:
import scipy.stats as st
import scipy.special
from scipy.stats import poisson 
from scipy.special import factorial
from scipy.optimize import curve_fit

def fit_distribution(data):

    distributions = [poisson, st.norm, st.gamma, st.chi2, st.weibull_min]
    mles = []
    pars_list = []
    for distribution in distributions:

        if distribution == poisson:
            pars = np.mean(data)
            mle = -sum([math.log(i) for i in dist_poisson(data, np.mean(data))])

        else:
            pars = distribution.fit(data)
            mle = distribution.nnlf(pars, data)

        mles.append(mle)
        pars_list.append(pars)
        #print(pars)

    results = [(mles, pars_list) for mles,pars_list in zip(mles, pars_list)]
    best_fit = sorted(zip(distributions, mles), key=lambda d: d[1])[0]
    #print ('Best fit reached using {}, MLE value: {}'.format(best_fit[0].name, best_fit[1]))
    
    parameters = results[mles.index(min(mles))][1]
    #print(mles)
    return(str(poisson),np.mean(data) )
    #return best_fit, parameters
    

In [9]:
from bokeh.layouts import gridplot
from bokeh.plotting import figure, show, output_notebook
from scipy.stats import chi2
from scipy.stats import weibull_min
output_notebook()

def Plot_Distribution(Period_Frequency, data_fit, parameters):
    
    
    
    #create histogram 
    x_plot = np.linspace(0, 2.5*max(Period_Frequency) , 1000)
    entries, bin_edges = np.histogram(Period_Frequency, density=True, 
                                      bins=(max(Period_Frequency)+1),
                                      range=[-0.5, max(Period_Frequency)+0.5]) 

    
    # calculate binmiddles
    bin_middles = 0.5*(bin_edges[1:] + bin_edges[:-1])

    
    # poisson function, parameter lamb is the fit parameter
    def dist_poisson(k, lamb):
        return (lamb**k/factorial(k)) * np.exp(-lamb)
    
    
    # fit with curve_fit
    parameters, cov_matrix = curve_fit(dist_poisson, bin_middles, entries) 

    
    #CDF = poisson.cdf(x_plot, np.mean(Period_Frequency))
    
    
    #create Plot
    p1 = figure(title="{}".format(BD_PROC.PROCEDIMENTO.unique()[1]), background_fill_color="#E8DDCB")
    
    
    p1.quad(top=entries, bottom=0, left=bin_edges[:-1], right=bin_edges[1:],
            fill_color="#036564", line_color="#033649")
    
    p1.line(x_plot,  dist_poisson(x_plot, np.mean(Period_Frequency)),
            line_color="#D95B43", line_width=8, alpha=0.7,
            legend="{} | λ= {:.2f}".format("PDF", np.mean(Period_Frequency)))
    
    #p1.line(x_plot, CDF , line_color="white", line_width=2, alpha=0.7, legend="CDF")

    p1.legend.location = "top_center"
    p1.legend.background_fill_color = "darkgrey"
    p1.xaxis.axis_label = 'x'
    p1.yaxis.axis_label = 'Pr(x)'


    return gridplot(p1, ncols=1, plot_width=400, plot_height=400, toolbar_location=None)

Loading BokehJS ...

In [10]:
from bokeh.layouts import gridplot
from bokeh.plotting import figure


#Call Part 2B
#plot distribution for one material
procedimentos = sorted(BD_PED.COD_PROCEDIMENTO.unique())
fig=[]
for proc in procedimentos:
    BD_PROC = BD_PED[BD_PED.COD_PROCEDIMENTO == proc]
    Period_Frequency= Period_Setup()
    
    data_fit, parameters = fit_distribution(Period_Frequency)
    fig = Plot_Distribution(Period_Frequency, data_fit, parameters)
    if  proc==1: fig1=fig  
    if  proc==2: fig2=fig
    if  proc==3: fig3=fig
    if  proc==4: fig4=fig  

pic = gridplot([[fig1, fig2], [fig3, fig4]])
show(pic) 




In [11]:
def materials_dict():
    
    mat_dict = {}
    for proc in procedimentos:
        proc_dict = {}
        
        BD_PROC = BD_PED[BD_PED.COD_PROCEDIMENTO == proc]
        Material_list = BD_PROC.COD_MATERIAL.unique()
        #print("\n proc: {} lista: {},".format(proc,Material_list))
        for material in Material_list:
            
            BD_MAT = BD_PROC[BD_PROC.COD_MATERIAL == material]
            
            
            max_demand = BD_MAT.QTDE.max()
            f0 = 1-(BD_MAT.ATENDIMENTO.nunique() / BD_PROC.ATENDIMENTO.nunique())
            F = []
            F.append(f0)
            for j in range(1,max_demand+30):

                BD_QTDE = BD_MAT[BD_MAT.QTDE == j]
                f1 =  BD_QTDE.ATENDIMENTO.nunique()  /BD_PROC.ATENDIMENTO.nunique() #Verificar
                f1= f1/(1-f0)
                F.append(f1)
            
            
            
            proc_dict[material] = F

        
        mat_dict[proc] = proc_dict
        

    return mat_dict

In [12]:
def F_jk(j,k):
    f1_list = mat_dict[proc][cod_material]

    if j ==0:
        #print("j={} k={}".format(j,k))
        #print("f0: ",1)
        return 1
    
    if k==1:
        #print("j={} k={}".format(j,k))
        #print("f1: ",f1_list[j])
        return  f1_list[j]

    else:
        fjk = sum(  [ F_jk(i, k-1) * F_jk(j-i, 1)  for i in range (k-1, (j-1) +1)  ] )
        #print("j={} k={}".format(j,k))
        #print("fjk: ", fjk)
        return  fjk
    

In [13]:
def prob_distribution_mat( param_proc, proc, cod_material):
 
    upper_bound = math.ceil(12*param_proc)
    ceil_prob = 0.999

    param_proc = param_proc * (1 - mat_dict[proc][cod_material][0] ) #adjust lamb
    #print(param_proc)
    #print(proc)
    

    prob_list_proc = [dist_poisson(i,param_proc) for i in range(upper_bound) 
                        if (poisson.cdf(i,param_proc)<ceil_prob)]
    
    prob_list_proc.append(1-sum(prob_list_proc))
    
    lamb = np.mean(Period_Frequency)
    f1_list = mat_dict[proc][cod_material] #f0 inside
    parameters = lamb * sum([(j* f1_list[j] *(1 - f1_list[0]) ) for j in range(len(f1_list))])
    
    #print("lamb: ", lamb)
    #print("f1_list: ", f1_list)
    #print("list_proc: ",prob_list_proc)
    max_demand = len(mat_dict[proc][cod_material])-1
    
    
    prob_list_mat = []
    prob_list_mat.append(prob_list_proc[0])
    j=0
    while True:

        j+=1  
        prob_mat = sum ( [prob_list_proc[k] * F_jk(j,k) for  k in range(1,len(prob_list_proc))])
        prob_list_mat.append(prob_mat)
        
        
        if(sum(prob_list_mat)==1.0): 
            break
        if(sum(prob_list_mat)>.97 and sum(prob_list_mat)<=1.0): 
            prob_list_mat.append(1-sum(prob_list_mat))
            break
        if (sum(prob_list_mat)>1):
            prob_list_mat[-1] += (1-sum(prob_list_mat))
            break


            

    return prob_list_mat, parameters

In [14]:
#Call Part 2C
#get materials dist dict
mat_dict = materials_dict()
mat_dist_dict = {}
parameter_mat = {}

for proc in procedimentos:

    
    BD_PROC = BD_PED[BD_PED.COD_PROCEDIMENTO == proc]
    Period_Frequency= Period_Setup()
    data_fit, param_proc = fit_distribution(Period_Frequency)
    #print(param_proc)
  

    cod_list =  BD_PROC.COD_MATERIAL.unique()
    #print("\n proc: {} lista: {}".format(proc,cod_list))
    
    prov_dict = {}
    prov_param = {}
    for cod_material in cod_list:
        
        prob_list, parameters = prob_distribution_mat(param_proc, proc, cod_material)
        prov_dict[cod_material] = prob_list
        prov_param[cod_material] = parameters

        #print("cod: {}, list: {}".format(cod_material,prob_list))
        #break
    mat_dist_dict[proc] = prov_dict
    parameter_mat[proc] = prov_param
    #break

#parameter_mat

In [15]:
def get_uni_dict():

    uni_dict = {}
    uni_param = {}
    cod_list =  BD_PED.COD_MATERIAL.unique()
    
    for cod_material in cod_list:
        mat_prob_list = []
        mat_prob_list.append("NA")
        
        mat_param_list = []
        mat_param_list.append("NA")
        
        for proc in procedimentos:

            dist_proc = mat_dist_dict[proc]
            mat_param = parameter_mat[proc]
            
            mat_prob_list.append(dist_proc.get(cod_material, [0]))
            mat_param_list.append(mat_param.get(cod_material, 0))
            
            
            
            
            #break
        uni_dict[cod_material] = mat_prob_list
        uni_param[cod_material] = mat_param_list
        #break
    return uni_dict, uni_param


In [16]:
def get_weighted_dict(unif_dict, unif_param):
    weighted_dict = {}
    weighted_param = {}
    tot = 0
    sup=0
    cod_list =  BD_PED.COD_MATERIAL.unique()
    for cod_material in cod_list:
        uni_dict_prov = unif_dict.copy()
        uni_param_prov = unif_param.copy()
        #cod_material = 50770  
        
        
        
        qtde_proc = [0]* (len(procedimentos)+1)
        for proc in procedimentos:
            if (uni_dict_prov[cod_material][proc] != [0]):
                BD_PROC = BD_PED[BD_PED.COD_PROCEDIMENTO == proc]
                qtde_proc[proc] = BD_PROC.ATENDIMENTO.nunique()
        #print("QTDE REAL: ", qtde_proc)  

        
        max_len = max([0  if (uni_dict_prov[cod_material][proc])==[0]
                    else len(uni_dict_prov[cod_material][proc]) for proc in procedimentos])
        
        
        
        #print("max_len: ", max_len)
        prov_list ={}
        weighted_list = []
        prov_param ={}
        for proc in procedimentos:

                
                weighted_list=[0] *max_len
                uni_dict_prov[cod_material][proc] = uni_dict_prov[cod_material][proc]+ \
                                                (max_len - len(uni_dict_prov[cod_material][proc]))*[0]


                prov_list[proc] = [(uni_dict_prov[cod_material][proc][i]* (qtde_proc[proc]/ sum(qtde_proc)))
                                    for i in range(max_len)]
                
                
                prov_param[proc] = (uni_param_prov[cod_material][proc] * (qtde_proc[proc]/ sum(qtde_proc)))
                
                
                #print("prov_list: ", prov_list[proc])
                
                
        for i in range(max_len): 
            weighted_list[i] = sum([round(prov_list[proc][i],4) for proc in procedimentos])
            weighted_param[cod_material] = sum([round(prov_param[proc],4) for proc in procedimentos])
        
             
        sup+=1
        #print("CODIGO: ", cod_material)
        #print("QTDE: ", qtde_proc)
        
        if (round(sum(weighted_list)) !=1):
            print("ERRO:", cod_material )
            print("QTDE: ", qtde_proc)
            print("soma: ", sum(weighted_list))
            print("weighted list: ",weighted_list)
            print("###############\n")
            tot+=1

            
        
        #else:
            #print("Acertou: ", cod_material)
            #print("QTDE: ", qtde_proc)
            #print("###############\n")
        #break
        weighted_dict[cod_material] = weighted_list
    #print("Erros: ", tot)
    #print("total: ", sup)
    return weighted_dict,weighted_param

In [34]:
#CALL part 2D - Unifying dicts
uni_dict, uni_param  = get_uni_dict()
weighted_dict, weighted_param = get_weighted_dict(uni_dict, uni_param)
#weighted_dict
#weighted_param

# Modelo


1. Estados
    1. Nível de estoque
    
    
2. Ações
    1. Pedir 0 a (12 * μ_semanal) *1.25

    
    
3. Cadeia de Markov Absorvente (MDP tempo Finito)
    1. Programacao Indutiva? (qual foi usado em aula?)



4. FALTA FAZER A FUNCAO DE CUSTO


In [48]:
def transition_matrix(states, policy):
    

    Mat=np.zeros((len(states),len(states)))
    mc= []
    s = policy[0]
    S = policy[1]
    backorder = policy[2]
    
        
    #print("s: ", s)
    #print("S: ", S)       
    #print("bo: ", backorder)
    #print("states: ", states)
    #print("MAT: ", Mat)
    
    
    for row_index in states: 
               
        #fill row_prov with zeros
        row_prov=[0] * (max(states)+1) 
        #print("len:", len(row_prov))
        ini = row_index
         
        if(ini<s):
            ini = S-backorder
            fim = S+1
            #print("size1 bo:", fim-ini)
            #print("size2 bo:",len(prob_list))
            #print("ini:", ini)
            #print("fim:", fim)
            row_prov[ini:fim] = prob_list[::-1]
            #print("row_prov : ", row_prov)       

        else:
            ini = row_index - backorder
            fim = row_index+1
            
            if(ini<0): #must trunc
                
                prov = prob_list.copy()
                #print("prov_trunc: ", prov)
                for i in range(abs(ini)+1):
                    row_prov[0] += prov.pop()
                    #print("row_prov 0 : ", row_prov[0])
                prov.reverse()
                row_prov[1:fim] = prov
                #print("row_prov : ", row_prov)
                
            else:
                #print("size1:",fim-ini)
                #print("size2:",len(prob_list))
                #print(row_index)
                row_prov[ini:fim] = prob_list[::-1]
            
        Mat[row_index] = row_prov[:]
        #print(row_prov)
    #print(Mat)
    return Mat



In [49]:
def average_cost(states, policy, A, h, b): #25/3/200
    s = policy[0]
    S = policy[1]
    backorder = policy[2]
    
    policy_cost = {}
    
    for state in states:
        if(state < backorder):
            policy_cost[state] = A + (backorder-state)*b
                
        elif((state >= backorder) and (state < s)):
            policy_cost[state] = A + (state-backorder)*h
    
        else: 
            policy_cost[state] =  (state-backorder)* h
    
    return policy_cost


    

In [50]:
def reward_dict(states, actions, A, h, b): 


    reward_dict = {}
    bo_lv= period_range
    
    for state in states:
        row_prov = {}
        keys = []
        if state < bo_lv: 
            keys = actions[:]
            for i in keys:
                row_prov[i] = (-A + (state-bo_lv)*b)*(-1)*(-1)
            
        
        elif(state >= (max(actions) + bo_lv )):
            keys = [0]
            row_prov[0] = (state-bo_lv)*h*(-1)
            
            
        else:
            fim = actions[-1] - (state-bo_lv) +1
            keys = actions[:fim]
            
            for i in keys:
                if i == 0:
                    row_prov[i] = (state-bo_lv)*h*(-1)
                else:
                    row_prov[i] = ((state-bo_lv) + A)*(-1)
                
        #print("\n estado: ", state) 
        #print(row_prov)       
        reward_dict[state] = row_prov
            
    return reward_dict




In [51]:
def Actions_dict(states, actions):

    actions_dict = {}
    bo_lv= period_range
    
    for state in states:
        row_prov = []
        keys = []
        if state < bo_lv: 
            keys = actions[(bo_lv - state)::]
            for i in keys:
                row_prov.append(i)
            
        
        elif(state >= (max(actions) + bo_lv )):
            keys = [0]
            row_prov.append(0)
            
            
        else:
            fim = actions[-1] - (state-bo_lv) +1
            keys = actions[:fim]
            
            for i in keys:
                if i == 0:
                     row_prov.append(i)
                else:
                     row_prov.append(i)
        actions_dict[state] = row_prov
    #print(actions_dict)                   
    return actions_dict




def T(state, action, index):

    s1 = [i for i in range(state+action-period_range,state+action+1)]
    #print("action: ", action)
    #print("state: ", state)
    #print("s1: ", s1)
    
    period = list(range(period_range+1))
    p = [prob_list[i] for i in period[::-1] ]       
    #print("perc: ",p)
    #print("index: ",index)
    #print("##########################\n")
    return p[index], s1[index]



def value_iteration(states, R, actions):
    epsilon=0.001
    gamma =.9
    U1 = dict([(s, 0) for s in states])
    Act = dict([(s, 0) for s in states])
    period = list(range(period_range+1))
    A = Actions_dict(states, actions)
    #print(A)
    it=0
    while True:
        it+=1
        #print("iter:", iter)
        U = U1.copy()
        delta_max, delta_min = (0,0)
   
        

        for s in states:  #verificar dicionarios R e A

            m = [ R[s][a] +  sum([T(s, a, s1)[0]* U[T(s, a, s1)[1]] for s1 in period]) for a in A[s]]
                                         
            U1[s] = max(m)
            #print("estado", s)
            #print("acao\n", m)
            


            delta_max = max(delta_max, abs(U1[s] - U[s]))
            delta_min = min(delta_min, abs(U1[s] - U[s]))
            Act[s] = A[s][np.argmax(m)]
            
            print("iter: {} , Un({}) = {}".format(it, s, m))
            
           
        print("eps: {} , d_max: {}, d_min: {}".format(delta_max - delta_min, delta_max,delta_min))
        print("U1(s): ", U1) 
        print("\n############################################\n")
        if (delta_max - delta_min) < epsilon:
            #print("ACT: ", Act)
            return U1, Act
        
        
        if it>400:
            print("ITV Failed")
            return 0,0
        
             

In [53]:
#CALL Functions Part3A:
from discreteMarkovChain import markovChain


def CALL_A(prob_list, cod_material, parameters,A, h, b):
    #States: Matriz with size of 125% of 12 week mean
    period_range = len(prob_list)-1
    bo_lv = period_range
    num_states = max(  int(math.ceil(52*parameters)) + bo_lv   ,4+bo_lv)
    states = np.arange(num_states)

    
    #Creating a transition matrix for my trial policy
    trial_policy = (bo_lv+1, bo_lv*2 +1, bo_lv) 
    T_matrix = transition_matrix(states, trial_policy)
    #print("states: ", states)
    #print("policy: ", trial_policy)
    
    
    #plot steady states distribution
    Steady_States = markovChain(T_matrix)
    Steady_States.computePi('linear') #We can also use 'power', 'krylov' or 'eigen'
    Pi = Steady_States.pi
    #print("Pi: ",Pi)

    
    #Calculating Avg_cost for trial policy
    state_cost = average_cost(states, trial_policy, A, h, b)
    #print("state_cost: ",state_cost)
    
    
    policy_cost=0
    for i in states:
        policy_cost += Pi[i] * state_cost[i]
    

    policy = (1,bo_lv*1+1) #Caso Particular
    print ('Para A = {}, h = {}, b = {}, com política{}, o custo foi: {} '.format(A, h, b, policy, policy_cost))
    

         #Plot heatmap and Steady States dist:
    #COMO PLOTAR OS DOIS JUNTOS + Pandas???
    #Qual o cod teste?? Caso Particular
    
    #if(cod_material == 52811):
        #sns.heatmap(T_matrix)
        #plt.plot(Steady_States.pi)
    
       
    
    return period_range, states, policy_cost


In [54]:
#CALL Functions Part3B:

def CALL_B(period_range, states, parameters, A, h, b):

    
    #defining range of actions
    bo_lv = period_range
    max_action = max(  int(math.ceil(52*parameters)) ,  4)
    actions = range(max_action)
    

    #Running Value Iteration Algorithm
    R = reward_dict(states, actions,A, h, b)
    U,Act = value_iteration(states, R, actions)

    #Get my best policy for a trial paramter (A,h,b)
    S = Act[0] - period_range
    s = list(Act.keys())[list(Act.values()).index(0)] -  1
    policy = (s,S)
    

 
    #Creating a transition matrix for best policy
    best_policy = (bo_lv+s,bo_lv+ S, bo_lv) 
    #print("best policy: ",best_policy)
    #print("new policy: ", best_policy)
    #print("states: ", states)
    T_matrix = transition_matrix(states, best_policy)


    #Calculating Steady State
    Steady_States = markovChain(T_matrix)
    Steady_States.computePi('linear')
    Pi = Steady_States.pi

    state_cost = average_cost(states, best_policy, A, h, b)
    policy_cost=0
    for i in states:
        policy_cost += Pi[i] * state_cost[i]


    print ('Para A = {}, h = {}, b = {}, com política{}, o custo foi: {} '.format(A, h, b, policy, policy_cost))


    
    return policy, policy_cost, Act


In [55]:
#CALL Part 3C - Concat

cod_list =  BD_PED.COD_MATERIAL.unique()

A,h,b = (25,1,100) #Caso Particular
itera=0
for cod_material in cod_list:
    prob_list=[]
    
    #cod_material = 54176
    print(cod_material)
    prob_list = weighted_dict[cod_material]
    parameters = weighted_param[cod_material]
    #print(prob_list)
    
    #prob_list = [0.18,0.31, 0.27, 0.15, 0.06, 0.02, 0.01]
    #parameters = 1.74
    
    
    period_range, states, policy_cost_trial = CALL_A(prob_list, cod_material, parameters, A, h, b)
    
    policy, policy_cost_best, Act = CALL_B(period_range, states, parameters, A, h, b)
    
    
    break
    
    
    print("######################################################")
    itera+=1
    if(itera>10):
        print("break")
        break
    
    #Armazenar resultados

52811
Para A = 25, h = 1, b = 100, com política(1, 3), o custo foi: 2.5267712767170107 
iter: 1 , Un(0) = [-225.0, -225.0]
iter: 1 , Un(1) = [-125.0, -125.0, -125.0]
iter: 1 , Un(2) = [0.0, -25.0, -25.0, -25.0]
iter: 1 , Un(3) = [-1.0, -26.0, -26.0]
iter: 1 , Un(4) = [-2.0, -27.0]
iter: 1 , Un(5) = [-3.0]
eps: 225.0 , d_max: 225.0, d_min: 0
U1(s):  {0: -225.0, 1: -125.0, 2: 0.0, 3: -1.0, 4: -2.0, 5: -3.0}

############################################

iter: 2 , Un(0) = [-232.765, -226.1769]
iter: 2 , Un(1) = [-132.765, -126.1769, -126.9375]
iter: 2 , Un(2) = [-7.765000000000001, -26.1769, -26.9375, -27.9375]
iter: 2 , Un(3) = [-2.1769, -27.9375, -28.9375]
iter: 2 , Un(4) = [-3.9375, -29.9375]
iter: 2 , Un(5) = [-5.9375]
eps: 7.765000000000001 , d_max: 7.765000000000001, d_min: 0
U1(s):  {0: -226.1769, 1: -126.1769, 2: -7.765000000000001, 3: -2.1769, 4: -3.9375, 5: -5.9375}

############################################

iter: 3 , Un(0) = [-240.13076114, -227.74052147]
iter: 3 , Un(1) = 

iter: 54 , Un(3) = [-124.09323546789906, -143.57902714230363, -154.0836576029071]
iter: 54 , Un(4) = [-119.57902714230363, -155.0836576029071]
iter: 54 , Un(5) = [-131.0836576029071]
eps: 2.31157447702941 , d_max: 2.31157447702941, d_min: 0
U1(s):  {0: -348.09323546789904, 1: -242.57902714230363, 2: -142.57902714230363, 3: -124.09323546789906, 4: -119.57902714230363, 5: -131.0836576029071}

############################################

iter: 55 , Un(0) = [-373.83950413812227, -350.403474443368]
iter: 55 , Un(1) = [-273.83950413812227, -250.403474443368, -244.88771117101606]
iter: 55 , Un(2) = [-148.83950413812227, -150.403474443368, -144.88771117101606, -155.39505399281316]
iter: 55 , Un(3) = [-126.40347444336798, -145.88771117101606, -156.39505399281316]
iter: 55 , Un(4) = [-121.88771117101608, -157.39505399281316]
iter: 55 , Un(5) = [-133.39505399281316]
eps: 2.311396389906065 , d_max: 2.311396389906065, d_min: 0
U1(s):  {0: -350.403474443368, 1: -244.88771117101606, 2: -144.88771117

iter: 98 , Un(0) = [-473.13945498872187, -449.71559275652294]
iter: 98 , Un(1) = [-373.13945498872187, -349.71559275652294, -344.18839920761366]
iter: 98 , Un(2) = [-248.13945498872187, -249.71559275652294, -244.18839920761368, -254.72425108101032]
iter: 98 , Un(3) = [-225.71559275652294, -245.18839920761368, -255.72425108101032]
iter: 98 , Un(4) = [-221.18839920761368, -256.7242510810103]
iter: 98 , Un(5) = [-232.72425108101032]
eps: 2.3095343961156516 , d_max: 2.3095343961156516, d_min: 0
U1(s):  {0: -449.71559275652294, 1: -344.18839920761366, 2: -244.18839920761368, 3: -225.71559275652294, 4: -221.18839920761368, 5: -232.72425108101032}

############################################

iter: 99 , Un(0) = [-475.44890087535657, -452.02504482745906]
iter: 99 , Un(1) = [-375.44890087535657, -352.02504482745906, -346.49784546893466]
iter: 99 , Un(2) = [-250.4489008753566, -252.02504482745903, -246.49784546893466, -257.03378012522546]
iter: 99 , Un(3) = [-228.02504482745903, -247.4978454689

iter: 113 , Un(2) = [-282.78116871605647, -284.3573506118117, -278.8301156085139, -289.366782360472]
iter: 113 , Un(3) = [-260.3573506118117, -279.8301156085139, -290.366782360472]
iter: 113 , Un(4) = [-255.83011560851392, -291.366782360472]
iter: 113 , Un(5) = [-267.366782360472]
eps: 2.3094817458645593 , d_max: 2.3094817458645593, d_min: 0
U1(s):  {0: -484.3573506118117, 1: -378.8301156085139, 2: -278.8301156085139, 3: -260.3573506118117, 4: -255.83011560851392, 5: -267.366782360472}

############################################

iter: 114 , Un(0) = [-510.0906173550202, -486.66680017061185]
iter: 114 , Un(1) = [-410.0906173550202, -386.66680017061185, -381.1395643032075]
iter: 114 , Un(2) = [-285.0906173550202, -286.66680017061185, -281.1395643032075, -291.6762621018096]
iter: 114 , Un(3) = [-262.66680017061185, -282.1395643032075, -292.6762621018096]
iter: 114 , Un(4) = [-258.1395643032075, -293.6762621018096]
iter: 114 , Un(5) = [-269.6762621018096]
eps: 2.3094797413376114 , d_max:

iter: 134 , Un(3) = [-308.85578555288674, -328.328543809156, -338.8655814700602]
iter: 134 , Un(4) = [-304.328543809156, -339.8655814700602]
iter: 134 , Un(5) = [-315.8655814700602]
eps: 2.309457805300781 , d_max: 2.309457805300781, d_min: 0
U1(s):  {0: -532.8557855528868, 1: -427.328543809156, 2: -327.328543809156, 3: -308.85578555288674, 4: -304.328543809156, 5: -315.8655814700602}

############################################

iter: 135 , Un(0) = [-558.589045568469, -535.1652347032167]
iter: 135 , Un(1) = [-458.58904556846903, -435.1652347032167, -429.63799289951294]
iter: 135 , Un(2) = [-333.58904556846903, -335.1652347032167, -329.63799289951294, -340.17503874712247]
iter: 135 , Un(3) = [-311.1652347032167, -330.63799289951294, -341.17503874712247]
iter: 135 , Un(4) = [-306.63799289951294, -342.17503874712247]
iter: 135 , Un(5) = [-318.17503874712247]
eps: 2.309457277062279 , d_max: 2.309457277062279, d_min: 0
U1(s):  {0: -535.1652347032167, 1: -429.63799289951294, 2: -329.6379928

...

iter: 284 , Un(0) = [-902.6969641820915, -879.2731537881267]
iter: 284 , Un(1) = [-802.6969641820915, -779.2731537881267, -773.7459115416891]
iter: 284 , Un(2) = [-677.6969641820915, -679.2731537881267, -673.7459115416891, -684.2830838142027]
iter: 284 , Un(3) = [-655.2731537881267, -674.7459115416891, -685.2830838142027]
iter: 284 , Un(4) = [-650.7459115416891, -686.2830838142027]
iter: 284 , Un(5) = [-662.2830838142027]
eps: 2.3094491205982877 , d_max: 2.3094491205982877, d_min: 0
U1(s):  {0: -879.2731537881267, 1: -773.7459115416891, 2: -673.7459115416891, 3: -655.2731537881267, 4: -650.7459115416891, 5: -662.2830838142027}

############################################

iter: 285 , Un(0) = [-905.0064133019573, -881.5826029079926]
iter: 285 , Un(1) = [-805.0064133019573, -781.5826029079926, -776.055360661555]
iter: 285 , Un(2) = [-680.0064133019573, -681.5826029079926, -676.055360661555, -686.5925329347566]
iter: 285 , Un(3) = [-657.5826029079926, -677.055360661555, -687.592532934756

U1(s):  {0: -932.3904835450422, 1: -826.8632412986049, 2: -726.8632412986049, 3: -708.3904835450422, 4: -703.8632412986049, 5: -715.400413579776}

############################################

iter: 308 , Un(0) = [-958.1237430588732, -934.6999326649081]
iter: 308 , Un(1) = [-858.1237430588732, -834.6999326649081, -829.1726904184708]
iter: 308 , Un(2) = [-733.1237430588732, -734.6999326649081, -729.1726904184708, -739.7098626998053]
iter: 308 , Un(3) = [-710.6999326649081, -730.1726904184708, -740.7098626998053]
iter: 308 , Un(4) = [-706.1726904184708, -741.7098626998053]
iter: 308 , Un(5) = [-717.7098626998053]
eps: 2.309449120029285 , d_max: 2.309449120029285, d_min: 0
U1(s):  {0: -934.6999326649081, 1: -829.1726904184708, 2: -729.1726904184708, 3: -710.6999326649081, 4: -706.1726904184708, 5: -717.7098626998053}

############################################

iter: 309 , Un(0) = [-960.4331921787391, -937.0093817847741]
iter: 309 , Un(1) = [-860.4331921787391, -837.0093817847741, -831.

iter: 358 , Un(5) = [-833.1823186955213]
eps: 2.3094491198729656 , d_max: 2.3094491198729656, d_min: 0
U1(s):  {0: -1050.1723886582029, 1: -944.6451464117654, 2: -844.6451464117654, 3: -826.1723886582029, 4: -821.6451464117654, 5: -833.1823186955213}

############################################

iter: 359 , Un(0) = [-1075.9056481720336, -1052.481837778069]
iter: 359 , Un(1) = [-975.9056481720336, -952.4818377780688, -946.9545955316313]
iter: 359 , Un(2) = [-850.9056481720336, -852.4818377780688, -846.9545955316313, -857.4917678153939]
iter: 359 , Un(3) = [-828.4818377780688, -847.9545955316313, -858.4917678153939]
iter: 359 , Un(4) = [-823.9545955316313, -859.4917678153939]
iter: 359 , Un(5) = [-835.4917678153939]
eps: 2.3094491198726246 , d_max: 2.3094491198726246, d_min: 0
U1(s):  {0: -1052.481837778069, 1: -946.9545955316313, 2: -846.9545955316313, 3: -828.4818377780688, 4: -823.9545955316313, 5: -835.4917678153939}

############################################

iter: 360 , Un(0) =


############################################

iter: 384 , Un(0) = [-1133.641876168681, -1110.2180657747163]
iter: 384 , Un(1) = [-1033.641876168681, -1010.2180657747161, -1004.6908235282788]
iter: 384 , Un(2) = [-908.641876168681, -910.2180657747161, -904.6908235282788, -915.2279958121242]
iter: 384 , Un(3) = [-886.2180657747161, -905.6908235282788, -916.2279958121242]
iter: 384 , Un(4) = [-881.6908235282788, -917.2279958121242]
iter: 384 , Un(5) = [-893.2279958121242]
eps: 2.309449119867395 , d_max: 2.309449119867395, d_min: 0
U1(s):  {0: -1110.2180657747163, 1: -1004.6908235282788, 2: -904.6908235282788, 3: -886.2180657747161, 4: -881.6908235282788, 5: -893.2279958121242}

############################################

iter: 385 , Un(0) = [-1135.951325288547, -1112.527514894582]
iter: 385 , Un(1) = [-1035.951325288547, -1012.527514894582, -1007.0002726481447]
iter: 385 , Un(2) = [-910.951325288547, -912.527514894582, -907.0002726481447, -917.5374449319913]
iter: 385 , Un(3) = [-888.5

TypeError: 'int' object is not subscriptable

# Resultados


1. Estados
    1. Nível de estoque
    
    
2. Ações
    1. Pedir 0 a (12 * μ_semanal) *1.25

    
    
3. Cadeia de Markov Absorvente (MDP tempo Finito)
    1. Programacao Indutiva? (qual foi usado em aula?)



4. Artigo Guilherme (tese?)


In [ ]:
from graphviz import Digraph

import graphviz
L0= str(round(prob_list[0],2))+ "%"
L1= str(round(prob_list[1],2))+ "%"
L2= str(round(prob_list[2],2))+ "%"
L3= str(round(prob_list[3],2))+ "%"
L4= str(round(prob_list[4],2))+ "%"
L5= str(round(prob_list[5],2))+ "%"





f = Digraph('finite_state_machine', filename='fsm.gv')
f.attr(rankdir='LR', size='8,5')

f.attr('node', shape='doublecircle')
f.node('LR_0')
f.node('LR_1')

f.attr('node', shape='circle')

f.edge('LR_5', 'LR_0', label=L5,constraint='true')
f.edge('LR_5', 'LR_1', label=L4,constraint='true')
f.edge('LR_5', 'LR_2', label=L3,constraint='true')
f.edge('LR_5', 'LR_3', label=L2,constraint='true')
f.edge('LR_5', 'LR_4', label=L1,constraint='true')
f.edge('LR_5', 'LR_5', label=L0,constraint='true')



#f.edge('LR_1', 'LR_4', label=L0, constraint='true')
#f.edge('LR_1', 'LR_3', label=L1, constraint='true')
#f.edge('LR_1', 'LR_2', label=L2, constraint='true')
#f.edge('LR_1', 'LR_1', label=L3, constraint='true')
#f.edge('LR_1', 'LR_0', label=L4, constraint='true')


f

In [ ]:
def simulate_policies(states, action, period_range):
    iter = range(1,6)
    A_list = [25*i for i in iter]
    h_list = [1*i for i in iter]
    b_list= [100*i for i in iter]
    var_dict = {}
    for A in A_list:
        for h in h_list:
            for b in b_list:

                R = reward_dict(states, actions,A, h, b)
                U,Act = value_iteration(states, R)
                #print(Act)
                #print(period_range)
                S = Act[-period_range] - period_range
                s = list(Act.keys())[list(Act.values()).index(0)] -  1
                policy = (s,S)
                
                var_dict [(A,h,b)] = policy
                print ('Para A = {}, h = {}, b = {}, a política é: {}'.format(A, h, b, policy))
                
                
    set_policy=set(var_dict.values())
    return var_dict, set_policy

In [ ]:
def max_freq_policy(var_dict, set_policy):

    frequency = dict([(s, 0) for s in set_policy])

    for policy in set_policy:
        count = 0
        for policy_dict in var_dict.values():
            if policy == policy_dict:
                count +=  1
        frequency[policy] = count
    max_pol = max(frequency, key=frequency.get)
    #print(frequency)
    return max_pol, frequency


In [ ]:
#Call pt4


var_dict, set_policies = simulate_policies(states, actions, period_range)
max_pol, frequency_policies= max_freq_policy(var_dict, set_policies)

#map_policies = plot_map_policies(var_dict, set_policies)





In [ ]:
frequency_policies

In [ ]:
max_pol

In [ ]:
def plot_map_policies(var_dict, set_policies):
    
    return 0
    